In [1]:
import pandas as pd

In [45]:
df = pd.read_csv('R1supine_right hip_102373_2024-08-15 15-20-42.csv',skiprows = 99)
df_processed = pd.read_csv('R1supine_right hip_102373_2024-08-15 15-20-42_processed.csv',skiprows = 99)

In [18]:
df.rename(columns={'Unnamed: 1':'B','Unnamed: 2':'C','Unnamed: 3':'D'},inplace = True)

In [19]:
df.head()

,Unnamed: 0,B,C,D,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,2024-08-15 15:19:32:000,0.0202,-0.381,-0.9225,521,0,27.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-08-15 15:19:32:010,0.0202,-0.377,-0.9305,521,0,27.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-08-15 15:19:32:020,0.0242,-0.377,-0.9186,521,0,27.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-08-15 15:19:32:030,0.0242,-0.361,-0.9067,521,0,27.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-08-15 15:19:32:040,0.0122,-0.385,-0.9186,521,0,27.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df['Overall class'].value_counts()

right side         4031
left side           951
reclined            463
supine              302
upright             176
prone supported       3
Name: Overall class, dtype: int64

In [ ]:
SIGN(B101)*ACOS(- D101/SQRT(B101^2+D101^2))*180/PI()+180

ASIN(C101/(B101^2+C101^2+D101^2)^0.5)*180/3.14

IF(AND(N101>$S$5,N101<($S$5+180)),"supine-recline","prone-sit")

IF(AND(Q101="prone-sit",O101>0),"prone",IF(AND(Q101="prone-sit",O101>-23),"prone supported",IF(AND(Q101="prone-sit",O101>-63),"upright",IF(Q101="prone- sit","sitting",""))))

IF(AND(Q101="supine- recline",O101>15),"upsidedown",IF(AND(Q101="supine-recline",O101<- 36),"reclined",IF(AND(Q101="supine-recline",N101<($S$5+69)),"left side",IF(AND(Q101="supine-recline",N101>($S$5+101)),"right side",IF(Q101="supine-recline","supine","")))))

CONCATENATE(R101,S101)

In [23]:
import numpy as np
import pandas as pd

def excel_round(x, n):
    """Mimics Excel's ROUND function"""
    return np.round(x + 10**(-len(str(x))-1), n)

def diagnose_differences(df, S5):
    # Calculate intermediate results for comparison
    df['360 angle raw'] = np.sign(df['B']) * np.arccos(-df['D'] / np.sqrt(df['B']**2 + df['D']**2)) * 180 / np.pi + 180
    df['360 angle'] = excel_round(df['360 angle raw'], 2)
    
    df['Up/down angle raw'] = np.arcsin(df['C'] / np.sqrt(df['B']**2 + df['C']**2 + df['D']**2)) * 180 / np.pi
    df['Up/down angle'] = excel_round(df['Up/down angle raw'], 2)
    
    df['Body Rotation'] = np.where((df['360 angle'] > S5) & (df['360 angle'] < (S5 + 180)), 'supine-recline', 'prone-sit')
    
    prone_sit_conditions = [
        df['Up/down angle'] > 0,
        df['Up/down angle'] > -23,
        df['Up/down angle'] > -63
    ]
    prone_sit_choices = ['prone', 'prone supported', 'upright']
    df['Prone-sit class'] = np.select(prone_sit_conditions, prone_sit_choices, default='sitting')
    df.loc[df['Body Rotation'] != 'prone-sit', 'Prone-sit class'] = ""
    
    supine_recline_conditions = [
        df['Up/down angle'] > 15,
        df['Up/down angle'] < -36,
        df['360 angle'] < (S5 + 69),
        df['360 angle'] > (S5 + 101)
    ]
    supine_recline_choices = ['upsidedown', 'reclined', 'left side', 'right side']
    df['Supine-recline class'] = np.select(supine_recline_conditions, supine_recline_choices, default='supine')
    df.loc[df['Body Rotation'] != 'supine-recline', 'Supine-recline class'] = ""
    
    df['Overall class'] = (df['Prone-sit class'] + ' ' + df['Supine-recline class']).str.strip()
    
    return df

In [31]:
def excel_round(x, n):
    return np.round(x, n)

def diagnose_differences(df, S5):
    df['360 angle raw'] = np.sign(df['B']) * np.arccos(-df['D'] / np.sqrt(df['B']**2 + df['D']**2)) * 180 / np.pi + 180
    df['360 angle'] = excel_round(df['360 angle raw'], 2)
    
    df['Up/down angle raw'] = np.arcsin(df['C'] / np.sqrt(df['B']**2 + df['C']**2 + df['D']**2)) * 180 / np.pi
    df['Up/down angle'] = excel_round(df['Up/down angle raw'], 2)
    df['Body Rotation'] = np.where((df['360 angle'] > S5) & (df['360 angle'] < (S5 + 180)), 'supine-recline', 'prone-sit')
    
    prone_sit_conditions = [
        df['Up/down angle'] > 0,
        df['Up/down angle'] > -23,
        df['Up/down angle'] > -63
    ]
    prone_sit_choices = ['prone', 'prone supported', 'upright']
    df['Prone-sit class'] = np.select(prone_sit_conditions, prone_sit_choices, default='sitting')
    df.loc[df['Body Rotation'] != 'prone-sit', 'Prone-sit class'] = ""
    
    supine_recline_conditions = [
        df['Up/down angle'] > 15,
        df['Up/down angle'] < -36,
        df['360 angle'] < (S5 + 69),
        df['360 angle'] > (S5 + 101)
    ]
    supine_recline_choices = ['upsidedown', 'reclined', 'left side', 'right side']
    df['Supine-recline class'] = np.select(supine_recline_conditions, supine_recline_choices, default='supine')
    df.loc[df['Body Rotation'] != 'supine-recline', 'Supine-recline class'] = ""
    
    df['Overall class'] = (df['Prone-sit class'] + ' ' + df['Supine-recline class']).str.strip()
    
    return df

In [38]:
def diagnose_differences_raw(df, S5):
    df['360 angle raw'] = np.sign(df['B']) * np.arccos(-df['D'] / np.sqrt(df['B']**2 + df['D']**2)) * 180 / np.pi + 180
    df['360 angle'] = excel_round(df['360 angle raw'], 2)
    
    df['Up/down angle raw'] = np.arcsin(df['C'] / np.sqrt(df['B']**2 + df['C']**2 + df['D']**2)) * 180 / np.pi
    df['Up/down angle'] = excel_round(df['Up/down angle raw'], 2)
    df['Body Rotation'] = np.where((df['360 angle'] > S5) & (df['360 angle'] < (S5 + 180)), 'supine-recline', 'prone-sit')
    
    prone_sit_conditions = [
        df['Up/down angle raw'] > 0,
        df['Up/down angle raw'] > -23,
        df['Up/down angle raw'] > -63
    ]
    prone_sit_choices = ['prone', 'prone supported', 'upright']
    df['Prone-sit class'] = np.select(prone_sit_conditions, prone_sit_choices, default='sitting')
    df.loc[df['Body Rotation'] != 'prone-sit', 'Prone-sit class'] = ""
    
    supine_recline_conditions = [
        df['Up/down angle raw'] > 15,
        df['Up/down angle raw'] < -36,
        df['360 angle raw'] < (S5 + 69),
        df['360 angle raw'] > (S5 + 101)
    ]
    supine_recline_choices = ['upsidedown', 'reclined', 'left side', 'right side']
    df['Supine-recline class'] = np.select(supine_recline_conditions, supine_recline_choices, default='supine')
    df.loc[df['Body Rotation'] != 'supine-recline', 'Supine-recline class'] = ""
    
    df['Overall class'] = (df['Prone-sit class'] + ' ' + df['Supine-recline class']).str.strip()
    
    return df

In [32]:
df = pd.read_csv('R1supine_right hip_102373_2024-08-15 15-20-42_processed.csv',skiprows = 99)
df.rename(columns={'Unnamed: 1':'B','Unnamed: 2':'C','Unnamed: 3':'D'},inplace = True)
result_df = diagnose_differences(df, 140)
result_df

,Unnamed: 0,B,C,D,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Up down angle,Observed position,Body Rotation,Prone-sit class,Supine-recline class,Overall class,Unnamed: 20,360 angle raw,Up/down angle raw,Up/down angle
0,2024-08-15 15:19:32:000,0.0202,-0.381,-0.9225,521,0,27.8,NaN,NaN,NaN,...,-22.447558,NaN,supine-recline,,left side,left side,0.0,181.254406,-22.436178,-22.44
1,2024-08-15 15:19:32:010,0.0202,-0.377,-0.9305,521,0,27.8,NaN,NaN,NaN,...,-22.062237,NaN,supine-recline,,left side,left side,NaN,181.243625,-22.051052,-22.05
2,2024-08-15 15:19:32:020,0.0242,-0.377,-0.9186,521,0,27.8,NaN,NaN,NaN,...,-22.317931,NaN,supine-recline,,left side,left side,NaN,181.509076,-22.306616,-22.31
3,2024-08-15 15:19:32:030,0.0242,-0.361,-0.9067,521,0,27.8,NaN,NaN,NaN,...,-21.713830,NaN,supine-recline,,left side,left side,NaN,181.528873,-21.702822,-21.70
4,2024-08-15 15:19:32:040,0.0122,-0.385,-0.9186,521,0,27.8,NaN,NaN,NaN,...,-22.749071,NaN,supine-recline,,left side,left side,NaN,180.760905,-22.737539,-22.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,2024-08-15 15:20:31:950,-0.0079,-0.133,-0.0633,316,0,27.2,NaN,NaN,NaN,...,-64.408783,NaN,supine-recline,,reclined,reclined,NaN,172.886124,-64.376130,-64.38
5996,2024-08-15 15:20:31:960,-0.1203,-0.301,-0.2098,316,0,27.2,NaN,NaN,NaN,...,-51.245294,NaN,supine-recline,,reclined,reclined,NaN,150.169901,-51.219315,-51.22
5997,2024-08-15 15:20:31:970,-0.0279,-0.345,-0.4593,428,0,27.2,NaN,NaN,NaN,...,-36.879831,NaN,supine-recline,,reclined,reclined,NaN,176.523861,-36.861134,-36.86
5998,2024-08-15 15:20:31:980,-0.0520,-0.169,-0.3167,428,0,27.2,NaN,NaN,NaN,...,-27.784351,NaN,supine-recline,,left side,left side,NaN,170.675619,-27.770265,-27.77


In [25]:
result_df.columns

Index(['Unnamed: 0', 'B', 'C', 'D', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', '360 angle', 'Up down angle', 'Observed position',
       'Body Rotation', 'Prone-sit class', 'Supine-recline class',
       'Overall class', 'Unnamed: 20', '360 angle raw', 'Up/down angle raw',
       'Up/down angle'],
      dtype='object')

In [28]:
result_df['Overall class'].value_counts()

right side         4007
left side           951
reclined            463
supine              300
upright             176
sitting              74
upsidedown           26
prone supported       3
Name: Overall class, dtype: int64

In [29]:
result_df['Up/down angle raw'].value_counts()

 0.407663    16
 0.634968    15
 0.404840    15
 0.407267    14
 0.408044    14
             ..
-9.958357     1
-4.913877     1
 9.732378     1
 6.968414     1
 1.332305     1
Name: Up/down angle raw, Length: 4104, dtype: int64

In [30]:
result_df['Up/down angle'].value_counts()

 0.17     339
 0.41     298
 0.64     258
 0.40     231
-0.06     161
         ... 
-20.97      1
-22.51      1
-27.45      1
-28.49      1
-27.77      1
Name: Up/down angle, Length: 2247, dtype: int64

In [26]:
result_df['360 angle'].value_counts()

256.27    123
256.10    116
256.21    104
255.99     97
256.16     93
         ... 
214.70      1
233.25      1
232.02      1
225.41      1
157.04      1
Name: 360 angle, Length: 2384, dtype: int64

In [27]:
result_df['360 angle raw'].value_counts()

256.270269    88
256.100098    88
256.599197    74
255.875292    73
256.325341    72
              ..
251.468792     1
243.191686     1
245.900373     1
245.371479     1
157.040008     1
Name: 360 angle raw, Length: 2719, dtype: int64

In [33]:
result_df = diagnose_differences(df,140)

In [35]:
result_df['360 angle raw'].value_counts()

256.270269    88
256.100098    88
256.599197    74
255.875292    73
256.325341    72
              ..
251.468792     1
243.191686     1
245.900373     1
245.371479     1
157.040008     1
Name: 360 angle raw, Length: 2719, dtype: int64

In [36]:
result_df['360 angle'].value_counts()

256.27    123
256.10    116
256.21    104
255.99     97
256.16     93
         ... 
214.70      1
233.25      1
232.02      1
225.41      1
157.04      1
Name: 360 angle, Length: 2384, dtype: int64

In [37]:
result_df['Overall class'].value_counts()

right side         4007
left side           951
reclined            463
supine              300
upright             176
sitting              74
upsidedown           26
prone supported       3
Name: Overall class, dtype: int64

In [39]:
df = pd.read_csv('R1supine_right hip_102373_2024-08-15 15-20-42_processed.csv',skiprows = 99)
df.rename(columns={'Unnamed: 1':'B','Unnamed: 2':'C','Unnamed: 3':'D'},inplace = True)
result_df_raw = diagnose_differences_raw(df, 140)

In [41]:
result_df_raw['Overall class'].value_counts()

right side         4007
left side           951
reclined            463
supine              300
upright             176
sitting              74
upsidedown           26
prone supported       3
Name: Overall class, dtype: int64

In [ ]:
# result_df.to_csv('diagnostic_output.csv', index=False)

**Processed (Using Excel) vs Processed (Using Python)**

In [90]:
result_df

,Unnamed: 0,B,C,D,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Up down angle,Observed position,Body Rotation,Prone-sit class,Supine-recline class,Overall class,Unnamed: 20,360 angle raw,Up/down angle raw,Up/down angle
0,2024-08-15 15:19:32:000,0.0202,-0.381,-0.9225,521,0,27.8,NaN,NaN,NaN,...,-22.447558,NaN,supine-recline,,left side,left side,0.0,181.254406,-22.436178,-22.44
1,2024-08-15 15:19:32:010,0.0202,-0.377,-0.9305,521,0,27.8,NaN,NaN,NaN,...,-22.062237,NaN,supine-recline,,left side,left side,NaN,181.243625,-22.051052,-22.05
2,2024-08-15 15:19:32:020,0.0242,-0.377,-0.9186,521,0,27.8,NaN,NaN,NaN,...,-22.317931,NaN,supine-recline,,left side,left side,NaN,181.509076,-22.306616,-22.31
3,2024-08-15 15:19:32:030,0.0242,-0.361,-0.9067,521,0,27.8,NaN,NaN,NaN,...,-21.713830,NaN,supine-recline,,left side,left side,NaN,181.528873,-21.702822,-21.70
4,2024-08-15 15:19:32:040,0.0122,-0.385,-0.9186,521,0,27.8,NaN,NaN,NaN,...,-22.749071,NaN,supine-recline,,left side,left side,NaN,180.760905,-22.737539,-22.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,2024-08-15 15:20:31:950,-0.0079,-0.133,-0.0633,316,0,27.2,NaN,NaN,NaN,...,-64.408783,NaN,supine-recline,,reclined,reclined,NaN,172.886124,-64.376130,-64.38
5996,2024-08-15 15:20:31:960,-0.1203,-0.301,-0.2098,316,0,27.2,NaN,NaN,NaN,...,-51.245294,NaN,supine-recline,,reclined,reclined,NaN,150.169901,-51.219315,-51.22
5997,2024-08-15 15:20:31:970,-0.0279,-0.345,-0.4593,428,0,27.2,NaN,NaN,NaN,...,-36.879831,NaN,supine-recline,,reclined,reclined,NaN,176.523861,-36.861134,-36.86
5998,2024-08-15 15:20:31:980,-0.0520,-0.169,-0.3167,428,0,27.2,NaN,NaN,NaN,...,-27.784351,NaN,supine-recline,,left side,left side,NaN,170.675619,-27.770265,-27.77


In [91]:
result_df[result_df['Overall class']=='upsidedown'].index

Int64Index([1207, 1233, 1234, 1260, 1268, 1269, 1270, 1271, 1275, 1276, 1283,
            1289, 1290, 1291, 1292, 1315, 1341, 1343, 1344, 1357, 1360, 1376,
            1377, 1378, 1385, 1393],
           dtype='int64')

In [93]:
result_df.columns

Index(['Unnamed: 0', 'B', 'C', 'D', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', '360 angle', 'Up down angle', 'Observed position',
       'Body Rotation', 'Prone-sit class', 'Supine-recline class',
       'Overall class', 'Unnamed: 20', '360 angle raw', 'Up/down angle raw',
       'Up/down angle'],
      dtype='object')

In [92]:
result_df[result_df['Overall class']=='upsidedown'][['Body Rotation','']]

,Unnamed: 0,B,C,D,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Up down angle,Observed position,Body Rotation,Prone-sit class,Supine-recline class,Overall class,Unnamed: 20,360 angle raw,Up/down angle raw,Up/down angle
1207,2024-08-15 15:19:44:070,1.0239,0.3029,-0.0831,148,0,27.2,NaN,NaN,NaN,...,16.437020,NaN,supine-recline,,upsidedown,upsidedown,NaN,265.360029,16.428687,16.43
1233,2024-08-15 15:19:44:330,0.8834,0.2749,-0.1108,111,0,27.2,NaN,NaN,NaN,...,17.167555,NaN,supine-recline,,upsidedown,upsidedown,NaN,262.851037,17.158852,17.16
1234,2024-08-15 15:19:44:340,0.4699,2.3186,-0.4315,111,0,27.2,NaN,NaN,NaN,...,74.653574,NaN,supine-recline,,upsidedown,upsidedown,NaN,227.439348,74.615728,74.62
1260,2024-08-15 15:19:44:600,1.0520,0.7308,0.0357,0,0,27.2,NaN,NaN,NaN,...,34.789001,NaN,supine-recline,,upsidedown,upsidedown,NaN,271.943607,34.771364,34.77
1268,2024-08-15 15:19:44:680,0.9798,0.2909,-0.1306,0,0,27.2,NaN,NaN,NaN,...,16.407183,NaN,supine-recline,,upsidedown,upsidedown,NaN,262.407655,16.398865,16.40
1269,2024-08-15 15:19:44:690,0.9958,0.2989,-0.0554,0,0,27.2,NaN,NaN,NaN,...,16.691790,NaN,supine-recline,,upsidedown,upsidedown,NaN,266.815709,16.683328,16.68
1270,2024-08-15 15:19:44:700,0.9476,0.2749,0.0120,0,0,27.2,NaN,NaN,NaN,...,16.184489,NaN,supine-recline,,upsidedown,upsidedown,NaN,270.725530,16.176284,16.18
1271,2024-08-15 15:19:44:710,0.5301,0.9108,0.4040,0,0,27.2,NaN,NaN,NaN,...,53.831592,NaN,supine-recline,,upsidedown,upsidedown,NaN,307.311763,53.804302,53.80
1275,2024-08-15 15:19:44:750,0.9476,0.2789,0.0674,0,0,27.2,NaN,NaN,NaN,...,16.369526,NaN,supine-recline,,upsidedown,upsidedown,NaN,274.068429,16.361227,16.36
1276,2024-08-15 15:19:44:760,1.1404,0.5469,0.9979,0,0,27.2,NaN,NaN,NaN,...,19.854796,NaN,supine-recline,,upsidedown,upsidedown,NaN,311.187348,19.844731,19.84


In [57]:
df_processed['Overall class'].isna().sum()

74

In [ ]:
df_processed['Overall class']

In [52]:
result_df['Overall class'].count()

6000

In [ ]:
df_processed

In [62]:
processed_excel = df_processed['Overall class'].value_counts().reset_index()

In [68]:
new_rows = pd.DataFrame({
    "index": ["upsidedown", "sitting"],
    "Overall class": [0, 0]
})

# Concatenating the new rows to the existing dataframe
processed_excel = pd.concat([processed_excel, new_rows], ignore_index=True)

In [64]:
processed_py = result_df['Overall class'].value_counts().reset_index()

In [66]:
processed_py

,index,Overall class
0,right side,4007
1,left side,951
2,reclined,463
3,supine,300
4,upright,176
5,sitting,74
6,upsidedown,26
7,prone supported,3


In [74]:
res = processed_py.merge(processed_excel,on='index',how='inner')

In [75]:
res

,index,Overall class_x,Overall class_y
0,right side,4007,4031
1,left side,951,951
2,reclined,463,463
3,supine,300,302
4,upright,176,176
5,sitting,74,0
6,upsidedown,26,0
7,prone supported,3,3


In [76]:
res['Difference'] = res['Overall class_y'] - res['Overall class_x']

In [78]:
res.drop(columns = ['Overall class_x','Overall class_y'],inplace = True)

In [79]:
res

,index,Difference
0,right side,24
1,left side,0
2,reclined,0
3,supine,2
4,upright,0
5,sitting,-74
6,upsidedown,-26
7,prone supported,0


# Test 2

##### Blank Row in Excel

In [95]:
result_df[result_df['Unnamed: 0']=='2024-08-15 15:20:26:040']

,Unnamed: 0,B,C,D,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Up down angle,Observed position,Body Rotation,Prone-sit class,Supine-recline class,Overall class,Unnamed: 20,360 angle raw,Up/down angle raw,Up/down angle
5404,2024-08-15 15:20:26:040,-0.1604,-0.8609,0.103,0,0,27.2,NaN,NaN,NaN,...,-77.554152,NaN,prone-sit,sitting,,sitting,NaN,57.293669,-77.514835,-77.51


In [99]:
excel_debug_blankrows = result_df[result_df['Prone-sit class']=='sitting'][['Unnamed: 0','B','C','D','Up down angle','Prone-sit class','Body Rotation','Supine-recline class','Overall class']]

In [102]:
excel_debug_blankrows.head()

,Unnamed: 0,B,C,D,Up down angle,Prone-sit class,Body Rotation,Supine-recline class,Overall class
5404,2024-08-15 15:20:26:040,-0.1604,-0.8609,0.1030,-77.554152,sitting,prone-sit,,sitting
5405,2024-08-15 15:20:26:050,-0.1685,-0.8049,-0.0791,-77.017639,sitting,prone-sit,,sitting
5419,2024-08-15 15:20:26:190,-0.7345,-1.4968,-0.1781,-63.241327,sitting,prone-sit,,sitting
5430,2024-08-15 15:20:26:300,-0.3572,-0.7849,-0.1623,-63.473369,sitting,prone-sit,,sitting
5431,2024-08-15 15:20:26:310,-0.2809,-0.8889,-0.2494,-67.125628,sitting,prone-sit,,sitting


In [101]:
excel_debug_blankrows['Supine-recline class'].unique()

array([''], dtype=object)

In [103]:
excel_debug_blankrows['Up down angle'].max()

-63.24132698

In [104]:
excel_debug_blankrows['Up down angle'].min()

-88.79019802

#### Fixed the blank rows and missing upsidedown in excel files. Validated that the updated webapp contains the correct values